# Benchmarking the N-Dimensional function in Numpy v.s. MyPyEigen

In [2]:
import MyPyEigen as pe
import numpy as np
import timeit, functools

In [3]:


def benchmark_function(func, args, number=10, preheat=5):
    """
    Benchmark 'func' with arguments 'args'. Preheat the cache by calling the function
    'preheat' times. Then, call 'func' 'number' times and return the average execution time in seconds.
    """
    for _ in range(preheat):
        func(*args)
    return timeit.timeit(lambda: func(*args), number=number) / number

# Ensure reproducibility
np.random.seed(12345)

# For ND append, we work with 3D arrays.
# Let's use arrays of shape (2, size, size) so that "append_nd" will double the size along axis 0.
sizes = [10, 50, 100, 500, 1000, 2000, 5000]
results_append_nd = {"numpy": [], "MyPyEigen": []}

for size in sizes:
    # Create two 3D arrays with shape (2, size, size)
    a = np.random.rand(2, size, size).astype(np.float64)
    b = np.random.rand(2, size, size).astype(np.float64)
    
    # For NumPy: use np.concatenate along axis 0.
    t_numpy = benchmark_function(lambda a, b: np.concatenate([a, b], axis=0), (a, b), number=10)
    results_append_nd["numpy"].append((size, t_numpy))
    
    # For MyPyEigen: call pe.append_nd (which is defined to take two 3D arrays and an axis)
    t_pe = benchmark_function(pe.append_nd, (a, b, 0), number=10)
    results_append_nd["MyPyEigen"].append((size, t_pe))

print("ND Append Benchmark:")
print("Size\tNumPy (sec)\tMyPyEigen (sec)")
for size, t in results_append_nd["numpy"]:
    t_pe = [x[1] for x in results_append_nd["MyPyEigen"] if x[0] == size][0]
    print(f"{size}\t{t:.9f}\t{t_pe:.9f}")


ND Append Benchmark:
Size	NumPy (sec)	MyPyEigen (sec)
10	0.000001104	0.000003533
50	0.000002692	0.000043721
100	0.000006500	0.000170621
500	0.000466579	0.005784825
1000	0.001955013	0.026430875
2000	0.010206029	0.128787558
5000	0.078665367	1.262826333


In [4]:
def benchmark_function(func, args, number=10, preheat=5):
    for _ in range(preheat):
        func(*args)
    return timeit.timeit(lambda: func(*args), number=number) / number

np.random.seed(12345)

# For ND concat, we work with 3D arrays of shape (2, size, size).
# Here we test by concatenating three arrays along axis 0.
sizes = [10, 50, 100, 500, 1000, 2000, 5000]
results_concat_nd = {"numpy": [], "MyPyEigen": []}

for size in sizes:
    A = np.random.rand(2, size, size).astype(np.float64)
    B = np.random.rand(2, size, size).astype(np.float64)
    C = np.random.rand(2, size, size).astype(np.float64)
    
    # For NumPy: use np.concatenate on a list of arrays along axis 0.
    t_numpy = benchmark_function(lambda lst: np.concatenate(lst, axis=0), ([A, B, C],), number=10)
    results_concat_nd["numpy"].append((size, t_numpy))
    
    # For MyPyEigen: call pe.concat_nd on the list of 3D arrays.
    t_pe = benchmark_function(pe.concat_nd, ([A, B, C], 0), number=10)
    results_concat_nd["MyPyEigen"].append((size, t_pe))

print("ND Concat Benchmark:")
print("Size\tNumPy (sec)\tMyPyEigen (sec)")
for size, t in results_concat_nd["numpy"]:
    t_pe = [x[1] for x in results_concat_nd["MyPyEigen"] if x[0] == size][0]
    print(f"{size}\t{t:.9f}\t{t_pe:.9f}")


ND Concat Benchmark:
Size	NumPy (sec)	MyPyEigen (sec)
10	0.000002304	0.000005304
50	0.000004871	0.000049537
100	0.000027554	0.000218367
500	0.000776862	0.007596646
1000	0.003280929	0.040678767
2000	0.017584588	0.174957229
5000	0.128864121	1.578376554


In [5]:
def benchmark_function(func, args, number=10, preheat=5):
    for _ in range(preheat):
        func(*args)
    return timeit.timeit(lambda: func(*args), number=number) / number

np.random.seed(12345)

# For ND stack, the input is a list of 2D arrays. The expected output is a 3D array.
sizes = [10, 50, 100, 500, 1000, 2000, 5000]
results_stack_nd = {"numpy": [], "MyPyEigen": []}

for size in sizes:
    # Create three 2D arrays with shape (size, size)
    A = np.random.rand(size, size).astype(np.float64)
    B = np.random.rand(size, size).astype(np.float64)
    C = np.random.rand(size, size).astype(np.float64)
    
    # For NumPy: use np.stack on the list along axis 0.
    t_numpy = benchmark_function(lambda lst: np.stack(lst, axis=0), ([A, B, C],), number=10)
    results_stack_nd["numpy"].append((size, t_numpy))
    
    # For MyPyEigen: call pe.stack_nd, which takes a list of 2D arrays and returns a 3D array.
    t_pe = benchmark_function(pe.stack_nd, ([A, B, C], 0), number=10)
    results_stack_nd["MyPyEigen"].append((size, t_pe))

print("ND Stack Benchmark:")
print("Size\tNumPy (sec)\tMyPyEigen (sec)")
for size, t in results_stack_nd["numpy"]:
    t_pe = [x[1] for x in results_stack_nd["MyPyEigen"] if x[0] == size][0]
    print(f"{size}\t{t:.9f}\t{t_pe:.9f}")


ND Stack Benchmark:
Size	NumPy (sec)	MyPyEigen (sec)
10	0.000003671	0.000004167
50	0.000019675	0.000045837
100	0.000007592	0.000143042
500	0.000392487	0.006530150
1000	0.001645538	0.029034200
2000	0.010266975	0.214569896
5000	0.054668883	2.079535983


## Result

### ND Append Benchmark
| Size | NumPy (sec)  | MyPyEigen (sec) | Speedup (MyPyEigen/Numpy) |
|------|--------------|------------------|---------------------------|
| 10   | 0.000001104  | 0.000003533      | 3.20x (Slower)            |
| 50   | 0.000002692  | 0.000043721      | 16.23x (Slower)           |
| 100  | 0.000006500  | 0.000170621      | 26.25x (Slower)           |
| 500  | 0.000466579  | 0.005784825      | 12.40x (Slower)           |
| 1000 | 0.001955013  | 0.026430875      | 13.52x (Slower)           |
| 2000 | 0.010206029  | 0.128787558      | 12.62x (Slower)           |
| 5000 | 0.078665367  | 1.262826333      | 16.05x (Slower)           |

### ND Concat Benchmark
| Size | NumPy (sec)  | MyPyEigen (sec) | Speedup (MyPyEigen/Numpy) |
|------|--------------|------------------|---------------------------|
| 10   | 0.000002304  | 0.000005304      | 2.30x (Slower)            |
| 50   | 0.000004871  | 0.000049537      | 10.17x (Slower)           |
| 100  | 0.000027554  | 0.000218367      | 7.92x (Slower)            |
| 500  | 0.000776862  | 0.007596646      | 9.78x (Slower)            |
| 1000 | 0.003280929  | 0.040678767      | 12.40x (Slower)           |
| 2000 | 0.017584588  | 0.174957229      | 9.95x (Slower)            |
| 5000 | 0.128864121  | 1.578376554      | 12.25x (Slower)           |

### ND Stack Benchmark
| Size | NumPy (sec)  | MyPyEigen (sec) | Speedup (MyPyEigen/Numpy) |
|------|--------------|------------------|---------------------------|
| 10   | 0.000003671  | 0.000004167      | 1.14x (Slower)            |
| 50   | 0.000019675  | 0.000045837      | 2.33x (Slower)            |
| 100  | 0.000007592  | 0.000143042      | 18.84x (Slower)           |
| 500  | 0.000392487  | 0.006530150      | 16.64x (Slower)           |
| 1000 | 0.001645538  | 0.029034200      | 17.65x (Slower)           |
| 2000 | 0.010266975  | 0.214569896      | 20.90x (Slower)           |
| 5000 | 0.054668883  | 2.079535983      | 38.04x (Slower)           |